In [ ]:
import pandas as pd 
import numpy as np

### 절대모멘텀
- 파일 로드 
- 파생변수 'STD-YM' 생성 -> Date 컬럼의 값을 년-월 추출 대입
- 년,월 별 마지막날 month_last_dt 데이터프레임 삽입 
- 전월의 종가의 값을 가지는 파생변수 생성
- 전년도의 종가의 값을 가지는 파생변수 생성
- 전월, 전년도의 종가를 가지고 거래내역 생성
- 수익율 계산

In [ ]:
df = pd.read_csv("../csv/SPY.csv")
df.head(1)

In [ ]:
## 새로운 파생변수 STD-YM 생성
## Date 컬럼을 기준으로 년-월 추출해서 파생변수 대입

## Date 컬럼을 데이터형을 datetime 변경 ( pd.to_datetime() )
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
## STD-YM 컬럼을 만들어서 Date컬럼에서 년(4자리)-월 추출
df['STD-YM'] = df['Date'].dt.strftime("%Y-%m")

In [ ]:
month_last_df = pd.DataFrame()

month_list = df['STD-YM'].unique()

for i in month_list:
    last_df = df[df['STD-YM'] == i].tail(1)
    month_last_df = pd.concat([month_last_df, last_df])
month_last_df.info()

In [ ]:
## 년-월 별 마지막날의 데이터들을 추출 
## 새로운 데이터프레임에 결합
month_last_df = df[~(df.shift(-1)['STD-YM'] == df['STD-YM'])]
month_last_df.info()

In [ ]:
## 'Date' 컬럼을 인덱스 변환
month_last_df.set_index(['Date'], inplace=True)

In [23]:
month_last_df.head(10)

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,26.706757,1003200,1993-01
1993-02-26,44.43750,44.43750,44.18750,44.40625,26.991682,66200,1993-02
1993-03-31,45.34375,45.46875,45.18750,45.18750,27.596252,111600,1993-03
1993-04-30,44.12500,44.28125,44.03125,44.03125,26.890142,88500,1993-04
1993-05-28,45.40625,45.40625,45.00000,45.21875,27.615356,79100,1993-05
1993-06-30,45.12500,45.21875,45.00000,45.06250,27.714964,437600,1993-06
1993-07-30,45.09375,45.09375,44.78125,44.84375,27.580431,75300,1993-07
1993-08-31,46.40625,46.56250,46.34375,46.56250,28.637531,66500,1993-08
1993-09-30,46.03125,46.12500,45.84375,45.93750,28.429131,99300,1993-09


In [25]:
## month_last_df 파생변수 2개 생성
## 'BF_1M' 전월의 종가(Adj Close)의 값
## 'BF_12M' 전년도의 종가(Adj Close)의 값
## NaN이 나온다면 0으로 대체
month_last_df["BF_1M"] = month_last_df.shift(1)['Adj Close'].fillna(0)
month_last_df["BF_12M"] = month_last_df.shift(12)['Adj Close'].fillna(0)
month_last_df.iloc[10:15]

C:\Users\moons\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\moons\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Open,High,Low,Close,Adj Close,Volume,STD-YM,BF_1M,BF_12M
Date,,,,,,,,,
1993-11-30,46.28125,46.5625,46.25000,46.34375,28.680576,230000,1993-11,28.990005,0.000000
1993-12-31,46.93750,47.0000,46.56250,46.59375,29.032679,312900,1993-12,28.680576,0.000000
1994-01-31,48.06250,48.3125,48.00000,48.21875,30.045223,313800,1994-01,29.032679,26.706757
1994-02-28,46.93750,47.0625,46.81250,46.81250,29.168985,333000,1994-02,30.045223,26.991682
1994-03-31,44.46875,44.6875,43.53125,44.59375,27.946764,788800,1994-03,29.168985,27.596252
